In [1]:
import pickle
from utils.analysis_from_interaction import *
from egg.core.language_analysis import Disent
from language_analysis_local import TopographicSimilarityConceptLevel, encode_target_concepts_for_topsim

# calculate metrics from stored interactions

In [2]:
control = False # whether original or control simulations are evaluated

if control:
    datasets = ('(4,8)', '(4,8)', '(4,8)', '(4,8)', '(4,8)', '(4,8)', '(4,8)')
    n_attributes = (4, 4, 4, 4, 4, 4, 4)
    n_values = (8, 8, 8, 8, 8, 8, 8)
    n_epochs = 300
    paths = ['results/(4,8)_sample_scaling_10_balanced_True_vsf_1/', 
             'results/(4,8)_sample_scaling_10_balanced_False_vsf_1/',
             'results/(4,8)_sample_scaling_10_balanced_True_vsf_2/', 
             'results/(4,8)_sample_scaling_10_balanced_False_vsf_2/',
             'results/(4,8)_sample_scaling_10_balanced_True_vsf_3/', 
             'results/(4,8)_sample_scaling_10_balanced_True_vsf_4/', 
             'results/(4,8)_sample_scaling_10_balanced_False_vsf_4/',]
    
else: 
    #datasets = ('(3,4)', '(3,8)', '(3,16)', '(4,4)', '(4,8)', '(5,4)')
    datasets = ('(3,4)', '(3,8)', '(4,4)', '(5,4)')
    #n_attributes = (3, 3, 3, 4, 4, 5)
    n_attributes = (3, 3, 4, 5)
    #n_values = (4, 8, 16, 4, 8, 4)
    n_values = (4, 8, 4, 4)
    #n_epochs = 300
    n_epochs = 100
    #paths = ['results/' + d + '_sample_scaling_10_balanced_False_vsf_3/' for d in datasets]
    paths = ['results/' + d + '_game_size_10_vsf_3/' for d in datasets]



### entropy scores: MI, effectiveness, efficiency

In [3]:
for d in range(len(datasets)):
    
    for run in range(5):

        path_to_run = paths[d] + 'standard/' + str(run) + '/'
        #path_to_interaction = (path_to_run + 'interactions/train/interactions_epoch' + str(n_epochs))
        path_to_interaction = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        interaction = torch.load(path_to_interaction)

        attributes = n_attributes[d]
        values = n_values[d]
        scores = information_scores(interaction, attributes, values, normalizer="arithmetic")
        
        pickle.dump(scores, open(path_to_run + 'entropy_scores.pkl', 'wb'))
    

###  message length

In [4]:
# we evaluated message length per hierarchy level after training but 
# you can also use the HierarchicalMessageLength callback and store the results 

for d in range(len(datasets)):
    
    for run in range(5): 
        
        path_to_run = paths[d] + 'standard/' + str(run) + '/'
        path_to_interaction = (path_to_run + 'interactions/train/interactions_epoch' + str(n_epochs))
        interaction = torch.load(path_to_interaction)

        attributes = n_attributes[d]
        values = n_values[d]
        scores = message_length_per_hierarchy_level(interaction, attributes)
        
        pickle.dump(scores, open(path_to_run + 'message_length_hierarchical.pkl', 'wb'))

FileNotFoundError: [Errno 2] No such file or directory: 'results/(3,4)_game_size_10_vsf_3/standard/0/interactions/train/interactions_epoch100'

###  symbol redundancy

In [ ]:
for d in range(len(datasets)):
    
    attributes = n_attributes[d]
    values = n_values[d]
    vs_factor = int(paths[d][-2])
    vocab_size = (n_values[d] + 1) * vs_factor + 1
    
    for run in range(5): 
        
        path_to_run = paths[d] + 'standard/' + str(run) + '/'
        symbol_f = np.load(path_to_run + 'symbols_pernsame.npy')
        path_to_interaction = (path_to_run + 'interactions/train/interactions_epoch' + str(n_epochs))
        interaction = torch.load(path_to_interaction)
        redundancy, MI = symbol_frequency(interaction, attributes, values, vocab_size)
        
        scores = {'symbol_redundancy': redundancy, 'MI_symbol-attribute_value': MI}
        
        pickle.dump(scores, open(path_to_run + 'symbol_redundancy.pkl', 'wb'))

###  compositionality scores: topsim, posdis, bosdis

In [6]:
# topsim
# although topsim values are stored throughout training if callbacks are verbose, we reevaluate the
# final topsim scores with more data points 

samples = 5000
for d, dataset in enumerate(datasets):
    
    dim = [n_values[d]]*n_attributes[d]
    
    for run in range(5):
        print("dataset", dataset, "run", run)
        
        topsim_final = {}
        path_to_run = paths[d] + 'standard/' + str(run) + '/'
        #path_to_interaction = (path_to_run + 'interactions/train/interactions_epoch' + str(n_epochs))
        path_to_interaction_train = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        path_to_interaction_test = (path_to_run + 'interactions/validation/epoch_' + str(n_epochs) + '/interaction_gpu0')
        
        TOPSIM = TopographicSimilarityConceptLevel(dim, is_gumbel=True)
        
        for mode in ['train', 'test']:

            if mode == 'train':
                interaction = torch.load(path_to_interaction_train)
            elif mode == 'test':
                interaction = torch.load(path_to_interaction_test)
                
                  
            messages = interaction.message.argmax(dim=-1)
            sender_input = interaction.sender_input
            n_targets = int(sender_input.shape[1]/2)
            # get target objects and fixed vectors to re-construct concepts
            target_objects = sender_input[:, :n_targets]
            target_objects = k_hot_to_attributes(target_objects, n_values[d])
            # concepts are defined by a list of target objects (here one sampled target object) and a fixed vector
            (objects, fixed) = retrieve_concepts_sampling(target_objects)
            # add one such that zero becomes an empty attribute for the calculation (_)
            objects = objects + 1
            concepts = torch.from_numpy(objects * (np.array(fixed)))
            specific_idx = np.where(np.sum(fixed, axis=1)==n_attributes[d])[0]
            #print(specific_idx)
            #messages_max_relevant = messages[max_relevant]
            #sender_input_max_relevant = sender_input[max_relevant]
            messages_specific = messages[specific_idx]
            concepts_specific = concepts[specific_idx]
            #print(messages_specific[:3])
            #print(concepts_specific[:3])

            messages = [msg.tolist() for msg in messages]
            messages_specific = [msg.tolist() for msg in messages_specific]

            #encoded_input = encode_input_for_topsim_hierarchical(sender_input, dim)
            #encoded_input_max_relevant = encode_input_for_topsim_hierarchical(sender_input_max_relevant, dim)
            encoded_input = encode_target_concepts_for_topsim(sender_input)
            topsim = TOPSIM.compute_topsim(encoded_input[0:samples], messages[0:samples])
            #topsim_max_relevant = TOPSIM.compute_topsim(encoded_input_max_relevant[0:samples], 
            #                                            messages_max_relevant[0:samples])
            topsim_specific = TOPSIM.compute_topsim(concepts_specific[0:samples], messages_specific[0:samples], 
                                                    meaning_distance_fn="edit")
            print('... topsim computed')

            topsim_final['topsim_' + mode] = topsim
            topsim_final['topsim_specific_' + mode] = topsim_specific
    
        pickle.dump(topsim_final, open(path_to_run +  "topsim_final.pkl", "wb" ) )
        print(topsim_final)        

dataset (3,4) run 0
... topsim computed
... topsim computed
{'topsim_train': 0.3201900600938201, 'topsim_specific_train': 0.3650747701056673, 'topsim_test': 0.32150253584605987, 'topsim_specific_test': 0.3813574301273949}
dataset (3,4) run 1
... topsim computed
... topsim computed
{'topsim_train': 0.35842271889793137, 'topsim_specific_train': 0.3905847256584553, 'topsim_test': 0.36294864973741675, 'topsim_specific_test': 0.4002057292759635}
dataset (3,4) run 2
... topsim computed
... topsim computed
{'topsim_train': 0.3891651401313913, 'topsim_specific_train': 0.40424826406258785, 'topsim_test': 0.4082981561833244, 'topsim_specific_test': 0.4322124223065069}
dataset (3,4) run 3
... topsim computed
... topsim computed
{'topsim_train': 0.3593255999034614, 'topsim_specific_train': 0.39646967404379807, 'topsim_test': 0.3563664337501424, 'topsim_specific_test': 0.4037038098752201}
dataset (3,4) run 4
... topsim computed
... topsim computed
{'topsim_train': 0.38661744646912566, 'topsim_speci

In [7]:
# use Disent callback from egg

for d in range(len(datasets)): 
    
    path = paths[d]
    dim = [n_values[d]] * n_attributes[d]
    n_features = n_attributes[d] * n_values[d]
    vs_factor = int(path[-2])
    vocab_size = (n_values[d] + 1) * vs_factor + 1
    
    print("data set", dim)
    
    for run in range(5):
        
        posdis_bosdis = {}
    
        path_to_run = paths[d] + 'standard/' + str(run) + '/'
        path_to_interaction_train = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        interaction = torch.load(path_to_interaction_train)
        
        messages = interaction.message.argmax(dim=-1)
        sender_input = interaction.sender_input
        n_targets = int(sender_input.shape[1]/2)
        # get target objects and fixed vectors to re-construct concepts
        target_objects = sender_input[:, :n_targets]
        target_objects = k_hot_to_attributes(target_objects, n_values[d])
        # concepts are defined by a list of target objects (here one sampled target object) and a fixed vector
        (objects, fixed) = retrieve_concepts_sampling(target_objects)
        # add one such that zero becomes an empty attribute for the calculation (_)
        objects = objects + 1
        concepts = torch.from_numpy(objects * (np.array(fixed)))

        # concrete/specific concepts: where all attributes are fixed
        #objects_max_relevance = torch.tensor(k_hot_to_attributes(
        #    objects[torch.sum(relevance_vectors, dim=1) == 0], dim[0]))
        #messages_max_relevance = messages[torch.sum(relevance_vectors, dim=1) == 0]
        #concepts_specific = torch.tensor(k_hot_to_attributes(
        #    objects[torch.sum(torch.from_numpy(fixed), dim=1) == n_attributes], dim[0]))
        concepts_specific = torch.tensor(
            objects[torch.sum(torch.from_numpy(fixed), dim=1) == n_attributes[d]])
        messages_specific = messages[torch.sum(torch.from_numpy(fixed), dim=1) == n_attributes[d]]
        
        #posdis_max_relevance = Disent.posdis(objects_max_relevance, messages_max_relevance)
        #bosdis_max_relevance = Disent.bosdis(objects_max_relevance, messages_max_relevance, vocab_size)
        posdis_specific = Disent.posdis(concepts_specific, messages_specific)
        bosdis_specific = Disent.bosdis(concepts_specific, messages_specific, vocab_size)
        
        #targets_encoded = torch.tensor(encode_target_concepts_for_topsim(sender_input))
        #objects = torch.tensor(k_hot_to_attributes(targets_encoded, n_values[d]+1))
        posdis = Disent.posdis(torch.from_numpy(objects), messages)
        bosdis = Disent.bosdis(torch.from_numpy(objects), messages, vocab_size)
        
        posdis_bosdis['posdis_specific'] = posdis_specific
        posdis_bosdis['bosdis_specific'] = bosdis_specific
        posdis_bosdis['posdis'] = posdis
        posdis_bosdis['bosdis'] = bosdis

        print(posdis_bosdis)
    
        pickle.dump(posdis_bosdis, open(path_to_run + "posdis_bosdis.pkl", "wb" ) )

data set [4, 4, 4]
{'posdis_specific': 0.07719212025403976, 'bosdis_specific': 0.16416414082050323, 'posdis': 0.08545204252004623, 'bosdis': 0.17468079924583435}
{'posdis_specific': 0.09685937315225601, 'bosdis_specific': 0.1584312915802002, 'posdis': 0.0938183069229126, 'bosdis': 0.1498732566833496}
{'posdis_specific': 0.052682965993881226, 'bosdis_specific': 0.25022318959236145, 'posdis': 0.05920030176639557, 'bosdis': 0.23529720306396484}
{'posdis_specific': 0.08278777450323105, 'bosdis_specific': 0.14884433150291443, 'posdis': 0.06403817981481552, 'bosdis': 0.1358392834663391}
{'posdis_specific': 0.12737058103084564, 'bosdis_specific': 0.3467981815338135, 'posdis': 0.09924725443124771, 'bosdis': 0.2873367369174957}
data set [8, 8, 8]
{'posdis_specific': 0.02495218813419342, 'bosdis_specific': 0.178904727101326, 'posdis': 0.02256193570792675, 'bosdis': 0.18862618505954742}
{'posdis_specific': 0.04120912030339241, 'bosdis_specific': 0.11425746977329254, 'posdis': 0.031247908249497414

### co-occurrences

In [10]:
for d in range(len(datasets)):
    
    vs_factor = int(paths[d][-2])
    
    for run in range(5): 
        
        path_to_run = paths[d] + 'standard/' + str(run) + '/'
        path_to_interaction = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        interaction = torch.load(path_to_interaction)

        attributes = n_attributes[d]
        values = n_values[d]
        
        scores = cooccurrence_per_hierarchy_level(interaction, attributes, values, vs_factor)

        print(scores)
        
        pickle.dump(scores, open(path_to_run + 'normalized_cooccurrence.pkl', 'wb'))
    

TypeError: only size-1 arrays can be converted to Python scalars